In [2]:
#!pip install -qU langchain-community
!pip install python-dotenv
!pip install --upgrade langchain_groq 
#!pip install --upgrade duckduckgo_search

                                              0.0/415.9 kB ? eta -:--:--
     --                                      30.7/415.9 kB 1.3 MB/s eta 0:00:01
     ------                                71.7/415.9 kB 787.7 kB/s eta 0:00:01
     -------------                          143.4/415.9 kB 1.1 MB/s eta 0:00:01
     --------------------                   225.3/415.9 kB 1.4 MB/s eta 0:00:01
     ------------------------------         337.9/415.9 kB 1.5 MB/s eta 0:00:01
     -------------------------------------- 415.9/415.9 kB 1.7 MB/s eta 0:00:00
                                              0.0/122.2 kB ? eta -:--:--
     -------------------------------------- 122.2/122.2 kB 3.6 MB/s eta 0:00:00
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: langchain

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cohere 5.13.3 requires numpy<2.0,>=1.26, but you have numpy 1.24.4 which is incompatible.
langchain 0.1.13 requires langchain-core<0.2.0,>=0.1.33, but you have langchain-core 0.3.45 which is incompatible.
langchain-cohere 0.1.5 requires langchain-core<0.3,>=0.1.42, but you have langchain-core 0.3.45 which is incompatible.
langchain-community 0.0.29 requires langchain-core<0.2.0,>=0.1.33, but you have langchain-core 0.3.45 which is incompatible.
langchain-openai 0.1.7 requires langchain-core<0.3,>=0.1.46, but you have langchain-core 0.3.45 which is incompatible.
langchain-text-splitters 0.0.2 requires langchain-core<0.3,>=0.1.28, but you have langchain-core 0.3.45 which is incompatible.
streamlit 1.35.0 requires pillow<11,>=7.1.0, but you have pillow 11.0.0 which is incompatible.
tensorflow-intel 2.13.0 requires nu

In [3]:
!pip install pymupdf

                                              0.0/16.6 MB ? eta -:--:--
                                              0.0/16.6 MB ? eta -:--:--
                                              0.1/16.6 MB 1.3 MB/s eta 0:00:13
                                              0.1/16.6 MB 1.1 MB/s eta 0:00:16
                                              0.2/16.6 MB 1.3 MB/s eta 0:00:13
                                              0.4/16.6 MB 1.7 MB/s eta 0:00:10
     -                                        0.6/16.6 MB 2.1 MB/s eta 0:00:08
     -                                        0.7/16.6 MB 2.3 MB/s eta 0:00:07
     --                                       0.9/16.6 MB 2.4 MB/s eta 0:00:07
     --                                       1.1/16.6 MB 2.6 MB/s eta 0:00:07
     --                                       1.2/16.6 MB 2.7 MB/s eta 0:00:06
     ---                                      1.4/16.6 MB 2.7 MB/s eta 0:00:06
     ---                                      1.5/16.6 MB 2.8 MB/s

In [5]:
!pip install pyaudio


                                              0.0/164.1 kB ? eta -:--:--
     -------                                 30.7/164.1 kB 1.4 MB/s eta 0:00:01
     ---------------------                   92.2/164.1 kB 1.1 MB/s eta 0:00:01
     -------------------------------------  163.8/164.1 kB 1.2 MB/s eta 0:00:01
     -------------------------------------- 164.1/164.1 kB 1.1 MB/s eta 0:00:00


In [6]:
import pyaudio
print(pyaudio.__version__)


0.2.14


In [10]:
!pip install speechrecognition
!pip install gtts

In [ ]:
import pymupdf as fitz
import json
import speech_recognition as sr  # For voice input
import threading  # For question timer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage
from gtts import gTTS
from IPython.display import Audio
import time
import os

# Initialize text-to-speech engine using gTTS
def speak(text, filename="output.mp3"):
    tts = gTTS(text=text, lang='en')
    tts.save(filename)
    return Audio(filename, autoplay=True)

# Load PDF and Extract All Text
def load_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

# Split Text into Manageable Chunks for LLM
def split_text(text, chunk_size=1000):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=100)
    return text_splitter.split_text(text)

# Generate Quiz for Each Text Chunk
def generate_quiz_from_chunk(llm, text_chunk, num_questions=1):
    prompt = f"""
    Based on the following content, create a {num_questions}-question multiple-choice quiz.
    Ensure that the correct answers are evenly distributed across all answer choices (A, B, C, and D),
    rather than favoring one letter. Make sure the distribution appears natural.

    Content:
    {text_chunk}

    Format the output as a valid JSON structure like this:
    {{
        "questions": [
            {{
                "question": "What is NLP?",
                "options": {{
                    "A": "Natural Language Processing",
                    "B": "Neural Language Processing",
                    "C": "New Language Processing",
                    "D": "None of the above"
                }},
                "answer": "A"
            }}
        ]
    }}

    Only return valid JSON, no extra text.
    """

    response = llm.invoke([HumanMessage(content=prompt)]).content

    try:
        quiz_data = json.loads(response)  # Ensure valid JSON
        return quiz_data.get("questions", [])  # Return only questions
    except json.JSONDecodeError:
        print("❌ Error: LLM did not return valid JSON.")
        return []

# Function to get voice input
def get_voice_input():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("🎤 Listening for answer... (Say A, B, C, or D)")
        speak("Listening for answer. Say A, B, C, or D.")
        recognizer.adjust_for_ambient_noise(source)
        try:
            audio = recognizer.listen(source, timeout=5)  # 5-second timeout
            text = recognizer.recognize_google(audio).strip().upper()
            if text in ["A", "B", "C", "D"]:
                return text
            else:
                print("❌ Unrecognized answer. Please try again.")
                speak("Unrecognized answer. Please try again.")
                return get_voice_input()
        except sr.UnknownValueError:
            print("❌ Could not understand audio. Please try again.")
            speak("Could not understand. Please try again.")
            return get_voice_input()
        except sr.RequestError:
            print("❌ Speech Recognition service unavailable.")
            speak("Speech recognition is not available.")
            return None
        except Exception as e:
            print(f"❌ Error: {e}")
            return None

# Function to get user input (keyboard or voice)
def get_user_input():
    print("Press 'V' to use voice input or type your answer (A, B, C, or D): ")
    speak("Press V to use voice input or type your answer.")
    choice = input().strip().upper()
    if choice == "V":
        return get_voice_input()
    return choice

# Conduct an Interactive Quiz with a Timer
def run_quiz(quiz_questions, time_limit=10):
    if not quiz_questions:
        print("❌ No quiz questions available.")
        return

    score = 0

    for q in quiz_questions:
        print("\n" + q["question"])  # Display question (No voice output for question)

        for key, value in q["options"].items():
            print(f"{key}: {value}")
            speak(f"{key}: {value}")  # Speak only the options

        user_answer = None
        timer = threading.Timer(time_limit, lambda: print("\n⏳ Time's up! Moving to next question..."))
        timer.start()

        user_answer = get_user_input()

        timer.cancel()  # Cancel timer when user answers

        while user_answer not in ["A", "B", "C", "D"]:
            print("❌ Invalid input. Please enter A, B, C, or D.")
            speak("Invalid input. Please enter A, B, C, or D.")
            user_answer = get_user_input()

        if user_answer == q["answer"]:
            print("✅ Correct!")
            speak("Correct!")
            score += 1
        else:
            print(f"❌ Incorrect. The correct answer is {q['answer']}: {q['options'][q['answer']]}")
            speak(f"Incorrect. The correct answer is {q['answer']}: {q['options'][q['answer']]}")

    print(f"\n🏆 Your final score: {score}/{len(quiz_questions)}")
    speak(f"Your final score is {score} out of {len(quiz_questions)}")

# Main Execution
pdf_path = "C:\\Users\\Windows 11\\Desktop\\Code\\Python\\Information_Lecture_1[1].pdf"
num_questions_per_chunk = 1  # Adjust as needed

# Load and process the PDF
text = load_pdf(pdf_path)
text_chunks = split_text(text, chunk_size=1000)  # Adjust chunk size as needed

# Initialize LLM
llm = ChatGroq(model="llama3-70b-8192", temperature=0.4,api_key = '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')

# Generate quiz questions for each chunk
all_questions = []
for chunk in text_chunks:
    quiz_questions = generate_quiz_from_chunk(llm, chunk, num_questions=num_questions_per_chunk)
    all_questions.extend(quiz_questions)  # Combine all questions

# Run the full quiz
run_quiz(all_questions, time_limit=10)  # 10 seconds per question



What is information according to the given definition?
A: Raw data with no meaning
B: Meaningfully interpreted data
C: A sequence of random symbols
D: A system that gathers data
Press 'V' to use voice input or type your answer (A, B, C, or D): 
🎤 Listening for answer... (Say A, B, C, or D)

⏳ Time's up! Moving to next question...
❌ Error: listening timed out while waiting for phrase to start
❌ Invalid input. Please enter A, B, C, or D.
Press 'V' to use voice input or type your answer (A, B, C, or D): 
❌ Incorrect. The correct answer is B: Meaningfully interpreted data

What is the main object of an information system?
A: To process an organization's data
B: To provide information to its users
C: To evaluate and analyze data
D: To produce meaningful and useful information
Press 'V' to use voice input or type your answer (A, B, C, or D): 
🎤 Listening for answer... (Say A, B, C, or D)

⏳ Time's up! Moving to next question...
❌ Error: listening timed out while waiting for phrase to start
